In [1]:
# Importamos las librerias que vamos a usar
import requests
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
import pandas as pd
import datetime


# Crear listas para guardar los datos
marcas = []
gas95s = []
gas98s = []
gasoils = []
marcas = []
direcciones = []
poblaciones = []
horarios = []

url = "https://www.clickgasoil.com/m/precio-de-gasolina-95-barcelona"

# Definimos la funcion que busca y recupera los enlaces de las gasolineras de Barcelona
def get_station_links(url):
    station_links = []
    try:
        page = requests.get(url)
    except HTTPError as e:
        print(e)
    except URLError:
        print("Server down or incorrect domain")
    else:
        res = BeautifulSoup(page.content,"html.parser")
        for link in res.find_all('a'):
            if link.has_attr('href'):
                if '/g/' in link.attrs['href']:
                    station_links.append(link.attrs['href'])
    return station_links

# Definimos la funcion que busca y recupera todos los datos referente a cada gasolinera
def get_info(station):
    try:
        html = ("https://www.clickgasoil.com" + station)
        page = requests.get(html)
    except HTTPError as e:
        print(e)
    except URLError:
        print("Server down or incorrect domain")
    else:
        # Estructuramos el contenido de la web obtenido para ser consultado facilmente
        soup = BeautifulSoup(page.content,"html.parser")
        
        # Buscamos la información general de cada gasolinera: Marca, Direcion, Poblacion, Horario
        info_gasolinera = soup.find("div", {"class":"column small-4 datos_gasolinera"})
        
        # Buscamos ahora las etiquetas div que hemos detectado, son las que engloban la información que buscamos
        div_lista = info_gasolinera.find_all('div')
        
        # Recorremos la información y guardamos los datos que nos interesan
        for i,dato in enumerate(div_lista):
            temp = dato.text.split(":")
            if (i==0): 
                marca = temp[1]
            if (i==1):
                direccion = temp[1]
            if (i==2):
                poblacion = temp[1]
            if (i==3):
                horario = temp[1] + temp[2]
                
        # Buscamos la tabla que contiene los precios de los combustibles a fecha de la consulta
        table = soup.find("table", { "class" : "small-12" })
        
        # Buscamos todas las filas de esta tabla y las guardamos en una lista
        td_list = table.find_all('td')
        
        # Recorremos la información y guardamos los datos que nos interesan
        for i,element in enumerate(td_list):
            if(i == 1):
                gasoil = td_list[1].text
            if (i == 7):
                gas95 = td_list[7].text
            if (i == 9):
                gas98 = td_list[9].text
            
        # Devolvemos los datos
        return marca,gasoil,gas95,gas98,direccion,poblacion,horario

def precios_gasolineras(station_links):
    # Para cada estacion, vamos a su pagina donde obtenemos todos sus datos y los precios del combustible del dia
    for station in station_links:
        marca,gasoil,gas95,gas98,direccion,poblacion,horario = get_info(station)

        # Guardamos los datos en listas que nos permitiran luego crear un dataframe que exportaremos.
        marcas.append(marca)
        gasoils.append(gasoil)
        gas95s.append(gas95)
        gas98s.append(gas98)
        direcciones.append(direccion)
        poblaciones.append(poblacion)
        horarios.append(horario)

    # Creamos un dataframe con todos los datos obtenidos
    df = pd.DataFrame(
        {'Marca':marcas,
         'Gasolina 95':gas95s,
         'Gasolina 98':gas98s,
         'Gasóleo A':gasoils,
         'Direccion':direcciones,
         'Poblacion':poblaciones,
         'Horario':horarios,
         'Fecha': datetime.datetime.now()})
    return df



# Parseando la web ClickGasoil que nos ofrece los precios de Gasóleo A, Gasolina 95 y Gasolina 98 de cada gasolinera de Barcelona
station_links = get_station_links(url)
df = precios_gasolineras(station_links)

print (df.head(10))

# Exportamos los datos en un archivos csv
df.to_csv("gasolineras_precios.csv", sep=',')

                  Marca Gasolina 95 Gasolina 98 Gasóleo A  \
0              PETROZAL      1.345€      1.399€    1.259€   
1               PREMOIL      1.215€           -    1.156€   
2                 CEPSA      1.389€      1.504€    1.289€   
3                 B-OIL      1.218€           -    1.138€   
4             EVOLUTION      1.239€      1.285€    1.157€   
5          M AUTOMOCION      1.356€      1.495€    1.286€   
6                 SARAS      1.329€      1.469€    1.229€   
7                  GALP      1.389€      1.499€    1.249€   
8                REPSOL      1.429€      1.569€    1.329€   
9  SHELL VILLA OLIMPICA      1.265€      1.369€    1.189€   

                                    Direccion  Poblacion  Horario  \
0                      PASSATGE MOÇANVIC, S/N  BARCELONA   L-V 06   
1                         CARRER PERE IV, 544  BARCELONA  L-D 24H   
2                 CALLE LOPE DE VEGA, 125/139  BARCELONA   L-D 06   
3                      CALLE BAC DE RODA, 130  BARCE